# MobileNet for Monkey Breed Classification 

In [3]:
from keras.applications import MobileNet

# MobileNet works on 224 x 224 pixel image input data
img_rows, img_cols = 224, 224

MobileNet = MobileNet(weights = 'imagenet', 
                      include_top = False,
                      input_shape = (img_rows, img_cols, 3))

for layer in MobileNet.layers:
    layer.trainable = False

In [4]:
def addTopModelMobileNet(bottom_model, num_classes):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024, activation = 'relu')(top_model)
    top_model = Dense(1024, activation = 'relu')(top_model)
    top_model = Dense(512, activation = 'relu')(top_model)
    top_model = Dense(num_classes, activation = 'softmax')(top_model)
    return top_model

In [5]:
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 10

FC_Head = addTopModelMobileNet(MobileNet, num_classes)

model = Model(inputs = MobileNet.input, outputs = FC_Head)

print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [8]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = './monkey_breed/train'
validation_data_dir = './monkey_breed/validation'

train_datagen = ImageDataGenerator(
        rescale = 1./255,
        rotation_range = 45,
        width_shift_range = 0.3,
        height_shift_range = 0.3,
        horizontal_flip = True,
        fill_mode = 'nearest')

validation_datagen = ImageDataGenerator(rescale = 1./255)

batch_size = 32

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size = (img_rows, img_cols),
        batch_size = batch_size,
        class_mode = 'categorical')

validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size = (img_rows, img_cols),
        batch_size = batch_size,
        class_mode = 'categorical')

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [14]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("./monkey_breed_mobileNet.h5",
                            monitor = 'val_loss',
                            mode = 'min',
                            save_best_only = True,
                            verbose = 1)

earlystop =  EarlyStopping(monitor =  'val_loss',
                          min_delta = 0,
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

callbacks = [earlystop, checkpoint]

model.compile(loss = 'categorical_crossentropy',
             optimizer  = RMSprop(lr =  0.001),
             metrics  = ['accuracy'])

nb_train_samples = 1098
nb_validation_samples = 272

epochs = 30
batch_size = 16

history = model.fit_generator(
                train_generator,
                steps_per_epoch =  nb_train_samples // batch_size,
                epochs = epochs,
                callbacks = callbacks,
                validation_data = validation_generator,
                validation_steps = nb_validation_samples // batch_size)



Epoch 1/30
68/68 [==============================] - 421s 6s/step - loss: 1.8722 - accuracy: 0.4822 - val_loss: 0.8503 - val_accuracy: 0.7765

Epoch 00001: val_loss improved from inf to 0.85030, saving model to ./monkey_breed_mobileNet.h5
Epoch 2/30
68/68 [==============================] - 389s 6s/step - loss: 0.6812 - accuracy: 0.7917 - val_loss: 1.4056 - val_accuracy: 0.7715

Epoch 00002: val_loss did not improve from 0.85030
Epoch 3/30
68/68 [==============================] - 373s 5s/step - loss: 0.5632 - accuracy: 0.8410 - val_loss: 0.2271 - val_accuracy: 0.9355

Epoch 00003: val_loss improved from 0.85030 to 0.22709, saving model to ./monkey_breed_mobileNet.h5
Epoch 4/30
68/68 [==============================] - 373s 5s/step - loss: 0.4427 - accuracy: 0.8743 - val_loss: 0.3821 - val_accuracy: 0.8770

Epoch 00004: val_loss did not improve from 0.22709
Epoch 5/30
68/68 [==============================] - 376s 6s/step - loss: 0.4591 - accuracy: 0.8748 - val_loss: 0.2902 - val_accuracy:

In [16]:
from keras.models import load_model

classifier = load_model("./monkey_breed_mobileNet.h5")

In [17]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

monkey_breeds_dict = {"[0]": "mantled_howler ", 
                      "[1]": "patas_monkey",
                      "[2]": "bald_uakari",
                      "[3]": "japanese_macaque",
                      "[4]": "pygmy_marmoset ",
                      "[5]": "white_headed_capuchin",
                      "[6]": "silvery_marmoset",
                      "[7]": "common_squirrel_monkey",
                      "[8]": "black_headed_night_monkey",
                      "[9]": "nilgiri_langur"}

monkey_breeds_dict_n = {"n0": "mantled_howler ", 
                      "n1": "patas_monkey",
                      "n2": "bald_uakari",
                      "n3": "japanese_macaque",
                      "n4": "pygmy_marmoset ",
                      "n5": "white_headed_capuchin",
                      "n6": "silvery_marmoset",
                      "n7": "common_squirrel_monkey",
                      "n8": "black_headed_night_monkey",
                      "n9": "nilgiri_langur"}

def draw_test(name, pred, im):
    monkey = monkey_breeds_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, monkey, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0, len(folders))
    path_class = folders[random_directory]
    print("Class - " + monkey_breeds_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0, len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path + "/" + image_name)    

for i in range(0,10):
    input_im = getRandomImage("./monkey_breed/validation/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx = 0.5, fy = 0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1, 224, 224, 3) 
    
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
   
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)

cv2.destroyAllWindows()

Class - white_headed_capuchin
Class - black_headed_night_monkey
Class - mantled_howler 
Class - bald_uakari
Class - common_squirrel_monkey
Class - black_headed_night_monkey
Class - white_headed_capuchin
Class - mantled_howler 
Class - silvery_marmoset
Class - nilgiri_langur
